Imports

In [261]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import math
from IPython import display

Experiment Set up

In [262]:
r_rad = 0.5
obs_rad = 0.5
r_pos = np.array([1.0,1.0])
r_vel = np.array([0.0,0.0]) #v,omega
r_heading = 0.0
linear_vel = np.array([r_vel[0]*math.cos(r_heading), r_vel[0]*math.sin(r_heading)])
goal = np.array([15.0,15.0])
obst_pos = np.array([7.5,7.5])
obst_vel = np.array([-1.0, -1.0])
dt = 0.1
v_max=  1.0
v_min = 0.2
w_cap = 0.5

goal_heading = math.atan2(goal[1]-r_pos[1],goal[0]-r_pos[0])

print(goal_heading)
rel_pos = obst_pos - r_pos
rel_vel = obst_vel - linear_vel
print(type(rel_vel))

0.7853981633974483
<class 'numpy.ndarray'>


Taylor Approximation of Velocity Obstacle Constraint

In [263]:
def taylor_approx(v,w,dt,a,r1,r2,r,R,head):
    t0=(v**2)*(r1*math.cos(head + a*dt) +r2*math.sin(head + a*dt))**2 + (v**2)*((r+R)**2 - (r1**2 + r2**2) )
    d = 2*(v**2)*(dt)*(r1*math.cos((head + a*dt)) + r2*math.sin(head + a*dt))*(-r1*math.sin(head + a*dt) + r2*math.cos(head + a*dt))*(w-a)
    return -(t0 + d)

Cost Function

In [264]:
def cost_fn(u, v_des, heading_des, v_cur, heading_cur, current_rel_p,current_rel_v,agent_r, obs_r,agent_v,dt ):
    # Store the list of function calls
    return (v_des - (v_cur[0] + u[0]))**2 + (heading_des - (heading_cur + (v_cur[1] + u[1])*dt)) **2

Constraints

In [265]:
def obt_avoidance_constraint(u, v_des, heading_des, v_cur, heading_cur, current_rel_p,current_rel_v,agent_r, obs_r,agent_v,dt):
    v0=(agent_v[0])*np.array([math.cos(0), math.sin(0)])
    v1=(agent_v[0] + u[0]) *np.array([math.cos((agent_v[1] + u[1])*dt), math.sin((agent_v[1] + u[1])*dt)])
    fut_rel_v =current_rel_v + v0 -v1 
    linear_vel = np.linalg.norm(current_rel_v)
    new_head = math.atan2(fut_rel_v[1],fut_rel_v[0])
    a = math.atan2(current_rel_v[1],current_rel_v[0])

    if (a>3 and new_head<0):
        new_head =new_head +2*math.pi 
    elif(a<(-3) and new_head>0 ) :
        new_head = new_head-2*math.pi
    ang_vel = (new_head-a)/dt
    r = current_rel_p
    return taylor_approx(linear_vel,ang_vel,dt,0,r[0],r[1],obs_r,agent_r,a)
    
    

Update

In [266]:
n = 1 
while np.linalg.norm(r_pos - goal) > 1.0:
    v_desired = v_max
    w_desired = goal_heading
    v_current = r_vel[0]
    w_current = r_vel[1]

    transform = np.array([[math.cos(r_heading), math.sin(r_heading)], 
                         [- math.sin(r_heading), math.cos(r_heading)]])

    rel_pos = np.transpose(np.matmul(transform,np.transpose(rel_pos)))
    rel_vel = np.transpose(np.matmul(transform,np.transpose(rel_vel)))
    


    arguments = (
    v_desired,
    w_desired,
    r_vel,
    r_heading,
    rel_pos,
    rel_vel,
    r_rad,
    obs_rad,
    r_vel,
    dt
)
    cons = ({'type': 'ineq', 'fun': obt_avoidance_constraint , 'args': arguments})

    sol = optimize.minimize(cost_fn, np.array([0, 0]), method="SLSQP", constraints = cons, args=arguments
                    )
    u_sol = sol['x']

    transform_back = np.array([[math.cos(r_heading), -math.sin(r_heading)], 
                         [math.sin(r_heading), math.cos(r_heading)]])

    rel_pos = np.transpose(np.matmul(transform_back,np.transpose(rel_pos)))
    rel_vel = np.transpose(np.matmul(transform_back,np.transpose(rel_vel)))


    r_vel =r_vel + np.array([u_sol[0], u_sol[1]])

    if r_vel[0]>v_max:
        r_vel[0] = v_max
    
    if r_vel[1]>w_cap:
        r_vel[1] = w_cap
    
    if r_vel[0]< v_min:
        r_vel[0] = v_min
    
    if r_vel[1] < -w_cap:
        r_vel[1] = -w_cap
    


    r_heading = r_heading + r_vel[1]*dt
    r_vel_2d = np.array([r_vel[0]*math.cos(r_heading), r_vel[0]*math.sin(r_heading)])
    
    r_pos = r_pos + r_vel_2d*dt
    obst_pos = obst_pos + obst_vel*dt
    rel_pos = obst_pos - r_pos
    goal_heading = math.atan2(goal[1]-r_pos[1],goal[0]-r_pos[0])

    rel_vel = obst_vel - r_vel_2d
    

    circle1 = plt.Circle((r_pos[0], r_pos[1]), 0.5, color='r')
    circle2 = plt.Circle((obst_pos[0], obst_pos[1]), 0.5, color='r')
    goal_plt = plt.Circle((15.0, 15.0), 0.1, color='g')
    

    
    ax = plt.gca()
    ax.cla() # clear things for fresh plot

    # change default range so that new circles will work
    ax.set_xlim((0, 20))
    ax.set_ylim((0, 20))

    ax.add_patch(circle1)
    ax.add_patch(circle2)
    ax.add_patch(goal_plt)
    ax.arrow(r_pos[0],r_pos[1], math.cos(r_heading),  math.sin(r_heading), head_width = 0.2, width = 0.05)

    plt.pause(0.1)
    display.clear_output(wait=True)
    display.display(plt.gcf())

IndexError: index 2 is out of bounds for axis 0 with size 2